In [ ]:
# !pip install -U -q transformers

In [1]:
# if error in BERT model init do
# !pip uninstall -y transformers accelerate
!pip install transformers accelerate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip show transformers

Name: transformers
Version: 4.28.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /pfs/data5/home/ma/ma_ma/ma_ndoering/.local/lib/python3.9/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, tokenizers, tqdm
Required-by: sentence-transformers


In [3]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
# !curl -X GET https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz -o video_games.tsv.gz

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.metrics import mean_squared_error
import torch
import transformers
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-32GB


# Import Data

In [7]:
train_df_balanced = pd.read_csv("./data/balanced/train_balanced.csv", on_bad_lines="skip")
val_df_balanced = pd.read_csv("./data/balanced/val_balanced.csv", on_bad_lines="skip")
train_df_stratified = pd.read_csv("./data/stratified/train_stratified.csv", on_bad_lines="skip")
val_df_stratified = pd.read_csv("./data/stratified/val_stratified.csv", on_bad_lines="skip")
test_df = pd.read_csv("./data/test.csv", on_bad_lines="skip")

In [8]:
# stratified data
X_train_stratified = np.array(train_df_stratified['full_review'])
y_train_stratified = np.array(train_df_stratified['star_rating']) -1 # need to substract 1 since BERT expects labels starting from

X_val_stratified = np.array(val_df_stratified['full_review'])
y_val_stratified = np.array(val_df_stratified['star_rating']) - 1

In [9]:
# balanced data
X_train = np.array(train_df_balanced['full_review'])
y_train = np.array(train_df_balanced['star_rating']) -1 # need to substract 1 since BERT expects labels starting from

X_val = np.array(val_df_balanced['full_review'])
y_val = np.array(val_df_balanced['star_rating']) - 1

In [10]:
# test data
X_test = np.array(test_df['full_review'])
y_test = np.array(test_df['star_rating']) - 1

# Tokenization

Create a class of type Dataset as an input for the Trainer function.

In [11]:
from transformers import BertTokenizer

In [12]:
import random
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

In [13]:
# Specify the pre-trained model name.
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased' # uncased means this tokenizer will also lower-case automatically

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Loading BERT tokenizer...


In [14]:
# We need the class of type Dataset for input in the Trainer function
# we can write a class: with the input texts, labels, tokenizer and max_len
# we'll have the full_review, input_ids, attention_mask, labels as our output
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

class AmazonReviewDataset(Dataset):

    def __init__(self, full_review, labels, tokenizer, max_len):
        self.full_review = full_review
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.full_review)

    def __getitem__(self, item):
        full_review = str(self.full_review[item])
        label = self.labels[item]
        # `encode_plus` will:
          #   (1) Tokenize the text.
          #   (2) Prepend the `[CLS]` token to the start.
          #   (3) Append the `[SEP]` token to the end.
          #   (4) Map tokens to their IDs.
          #   (5) Pad or truncate the sentence to `max_length` with [PAD] tokens
          #   (6) Create attention masks for [PAD] tokens
        encoding = self.tokenizer.encode_plus(
          full_review,
          add_special_tokens=True, # Add '[CLS]' and '[SEP]' and [PAD]
          max_length=self.max_len, # Pad & truncate all texts
          truncation=True,
          padding='max_length',
          return_token_type_ids=False,
          return_attention_mask=True, # Construct attention masks
          return_tensors='pt', # Return pytorch tensors
        )

        return {
          'news_text': full_review,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(label, dtype=torch.long)
        }

# BERT With Balanced Data

The following works with the balanced train and validation datasets.

## Hyperparameter Tuning

The following trains and fine-tunes a pre-defined BERT model. Moreover, optuna is used for hyperparameter tuning.

In [15]:
from transformers import TrainingArguments, Trainer, BertForSequenceClassification, BertConfig, get_linear_schedule_with_warmup
from torch.optim import Adam, AdamW, SGD
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

2023-05-25 16:49:57.032088: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-25 16:49:58.244641: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 16:50:20.043396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [16]:
import optuna

def objective(trial):
  # BERT model to tune
  # ------------------------------
  # PRE_TRAINED_MODEL_NAME = model

  # Hyperparameters
  num_train_epochs = trial.suggest_int("num_train_epochs", 2, 4)
  per_device_train_batch_size = trial.suggest_int("train_batch_size", 8, 32)
  per_device_eval_batch_size = trial.suggest_int("eval_batch_size", 3, 32)
  learning_rate = trial.suggest_float("learning_rate", 0.000001, 0.0001)

  training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=num_train_epochs, per_device_train_batch_size=per_device_train_batch_size, per_device_eval_batch_size=per_device_eval_batch_size, learning_rate=learning_rate)
  max_len = 256 # max number of token inputs

  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # bert outputs a raw representation of its confidence as "logits"
    probabilities = torch.nn.functional.softmax(torch.from_numpy(logits), dim=-1) # extract relative probabilites of class
    predicted_probabilities = torch.max(probabilities, axis=-1).values # get highest relative probability
    return { 'accuracy': accuracy_score(labels, predictions), 
            'recall_score': recall_score(labels, predictions, average='micro'), 
            'precision_score': precision_score(labels, predictions, average='micro'), 
            'f1_score': f1_score(labels, predictions, average='micro'), 
            #'highest_probability': predicted_probabilities 
            }

  model = BertForSequenceClassification.from_pretrained(
      PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
      num_labels = 5, # The number of output labels, in our case it's multi-class tasks with classes=5  
      output_attentions = False, # Whether the model returns attentions weights.
      output_hidden_states = False, # Whether the model returns all hidden-states.
  )

  # Experimentation with different optimzer ("adam", "sgd" etc) / scheduler combinations
  # add to "optimizers" parameter in Trainer class
  # ------------------------------------------------------------------------------------
  # optimizer = "adamw" # default in Trainer
  # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_train_steps) # default in trainer
  # optimizer = trial.suggest_categorical("optimizer", ["adamw", "adam", "sgd"])

  # Create the optimizer with the hyperparameter learning rate
#   if optimizer == "adamw":
#       optimizer_obj = AdamW(model.parameters(), lr=learning_rate)
#   elif optimizer == "adam":
#       optimizer_obj = Adam(model.parameters(), lr=learning_rate)
#   elif optimizer == "sgd":
#       optimizer_obj = SGD(model.parameters(), lr=learning_rate)


#   scheduler = get_linear_schedule_with_warmup(
#     optimizer_obj,
#     num_warmup_steps=trial.suggest_int("num_warmup_steps", 1, 10), # so könnte man diesen Parameter auch tunen, falls es da keine Vorgabewert gibt
#     num_training_steps=trial.suggest_int("num_training_steps", 1, 10) # so könnte man diesen Parameter auch tunen, falls es da keine Vorgabewert gibt
#   )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=AmazonReviewDataset(full_review=np.array(X_train), labels=np.array(y_train), tokenizer=tokenizer, max_len=max_len),
      eval_dataset=AmazonReviewDataset(full_review=np.array(X_val), labels=np.array(y_val), tokenizer=tokenizer, max_len=max_len),
      compute_metrics=compute_metrics,
      # optimizers = (optimizer_obj, scheduler) # uncomment if you want to use another optimizer scheduler than the default in the trainer
  )

  # training, evaluation and prediction loops can be done with one simple line of code:
  trainer.train()
  eval_metrics = trainer.evaluate()
  # pred_tuple = trainer.predict(AmazonReviewDataset(full_review=np.array(X_test), labels=np.array(y_test), tokenizer=tokenizer, max_len=max_len))

  #return pred_tuple, eval_metrics
  # Der Ausgabewert der Funktion darf nur ein numerischer Wert sein, also kein Tuple oder auch keine Liste von Metriken
  return eval_metrics['eval_f1_score']


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10) # je nachdem wie lange ein trial durchläuft kann man die Zahl auch erhöhen

# so kann man dann auf die trials zugreifen und die Hyperparameter mit dem jeweiligen Ergebnis anschauen
for trial in study.trials:
  hyperparams = trial.params
  result = trial.value
  print(f"Hyperparameters: {hyperparams}, Result: {result}")

# oder man greift direkt auf die erfolgreichste Hyperparameter Kombination zu
print(f"Best set of hyperparameters: {study.best_trial.params}, Result: {study.best_trial.value}")

[I 2023-05-24 13:48:04,215] A new study created in memory with name: no-name-b2ff879e-181b-496e-b504-7c09adc104cc
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

Epoch,Training Loss,Validation Loss,Accuracy,Recall Score,Precision Score,F1 Score
1,1.093600,1.010349,0.564556,0.564556,0.564556,0.564556
2,0.895700,0.999766,0.576444,0.576444,0.576444,0.576444
3,0.620200,1.159189,0.576889,0.576889,0.576889,0.576889


[I 2023-05-24 14:07:48,601] Trial 0 finished with value: 0.5768888888888889 and parameters: {'num_train_epochs': 3, 'train_batch_size': 17, 'eval_batch_size': 15, 'learning_rate': 9.839599241097951e-05}. Best is trial 0 with value: 0.5768888888888889.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the ch

Epoch,Training Loss,Validation Loss,Accuracy,Recall Score,Precision Score,F1 Score
1,1.167300,1.014193,0.563667,0.563667,0.563667,0.563667
2,0.905700,1.006043,0.573333,0.573333,0.573333,0.573333
3,0.658800,1.154715,0.573778,0.573778,0.573778,0.573778
4,0.413400,1.493700,0.573000,0.573000,0.573000,0.573000


[I 2023-05-24 14:31:46,253] Trial 1 finished with value: 0.573 and parameters: {'num_train_epochs': 4, 'train_batch_size': 22, 'eval_batch_size': 13, 'learning_rate': 9.913740913294872e-05}. Best is trial 0 with value: 0.5768888888888889.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a

...

Epoch,Training Loss,Validation Loss,Accuracy,Recall Score,Precision Score,F1 Score
1,1.005900,0.992094,0.576778,0.576778,0.576778,0.576778
2,0.876800,1.004888,0.582556,0.582556,0.582556,0.582556
3,0.678900,1.109909,0.583000,0.583000,0.583000,0.583000


[I 2023-05-24 16:56:41,769] Trial 8 finished with value: 0.583 and parameters: {'num_train_epochs': 3, 'train_batch_size': 8, 'eval_batch_size': 13, 'learning_rate': 1.9968164674180355e-05}. Best is trial 2 with value: 0.5883333333333334.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a

Epoch,Training Loss,Validation Loss,Accuracy,Recall Score,Precision Score,F1 Score
1,1.083900,1.020652,0.549556,0.549556,0.549556,0.549556
2,0.903500,1.016066,0.570111,0.570111,0.570111,0.570111
3,0.658200,1.137928,0.568667,0.568667,0.568667,0.568667
4,0.416400,1.455462,0.567444,0.567444,0.567444,0.567444


[I 2023-05-24 17:21:06,961] Trial 9 finished with value: 0.5674444444444444 and parameters: {'num_train_epochs': 4, 'train_batch_size': 19, 'eval_batch_size': 15, 'learning_rate': 8.511802356416834e-05}. Best is trial 2 with value: 0.5883333333333334.


Hyperparameters: {'num_train_epochs': 3, 'train_batch_size': 17, 'eval_batch_size': 15, 'learning_rate': 9.839599241097951e-05}, Result: 0.5768888888888889
Hyperparameters: {'num_train_epochs': 4, 'train_batch_size': 22, 'eval_batch_size': 13, 'learning_rate': 9.913740913294872e-05}, Result: 0.573
Hyperparameters: {'num_train_epochs': 2, 'train_batch_size': 28, 'eval_batch_size': 16, 'learning_rate': 8.346539548013792e-05}, Result: 0.5883333333333334
Hyperparameters: {'num_train_epochs': 3, 'train_batch_size': 11, 'eval_batch_size': 23, 'learning_rate': 2.5485765617944864e-05}, Result: 0.5834444444444444
Hyperparameters: {'num_train_epochs': 4, 'train_batch_size': 16, 'eval_batch_size': 4, 'learning_rate': 7.968599259152304e-06}, Result: 0.5807777777777777
Hyperparameters: {'num_train_epochs': 4, 'train_batch_size': 32, 'eval_batch_size': 8, 'learning_rate': 2.024766979022767e-05}, Result: 0.5821111111111111
Hyperparameters: {'num_train_epochs': 3, 'train_batch_size': 18, 'eval_batch_s

In [24]:
best_params = pd.DataFrame([study.best_trial.params])

In [26]:
best_params

,num_train_epochs,train_batch_size,eval_batch_size,learning_rate
0,2,28,16,0.000083


In [27]:
# Save the DataFrame to a CSV file
best_params.to_csv('hyperparameter_tuning/stratified_best_params.csv', index=False)

In [18]:
study.best_trial.values
# Best F1 result from tuning [0.5883333333333334]

[0.5883333333333334]

In [20]:
# Save the DataFrame to a CSV file
best_params.to_csv('hyperparameter_tuning/stratified_best_params.csv', index=False)

# Get the trials and their results
trials = study.trials
results = [{'trial_number': trial.number, 'value': trial.value, 'params': trial.params} for trial in trials]

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df.to_csv('stratified_hyperparameter_results.csv', index=False)

## Training

Training with the best hyperparameters received from the previous step.

In [19]:
def objective():
  # BERT model to tune
  # ------------------------------
  # PRE_TRAINED_MODEL_NAME = model

  # Hyperparameters
  num_train_epochs = 2
  per_device_train_batch_size = 28
  per_device_eval_batch_size = 16
  learning_rate = 8.346539548013792e-05

  training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=num_train_epochs, per_device_train_batch_size=per_device_train_batch_size, per_device_eval_batch_size=per_device_eval_batch_size, learning_rate=learning_rate)
  max_len = 256 # max number of token inputs

  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # bert outputs a raw representation of its confidence as "logits"
    probabilities = torch.nn.functional.softmax(torch.from_numpy(logits), dim=-1) # extract relative probabilites of class
    predicted_probabilities = torch.max(probabilities, axis=-1).values # get highest relative probability
    return { 'accuracy': accuracy_score(labels, predictions), 
            'recall_score': recall_score(labels, predictions, average='micro'), 
            'precision_score': precision_score(labels, predictions, average='micro'), 
            'f1_score': f1_score(labels, predictions, average='micro'), 
            'highest_probability': predicted_probabilities.tolist() 
            }

  model = BertForSequenceClassification.from_pretrained(
      PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
      num_labels = 5, # The number of output labels, in our case it's multi-class tasks with classes=5  
      output_attentions = False, # Whether the model returns attentions weights.
      output_hidden_states = False, # Whether the model returns all hidden-states.
  )

  # Experimentation with different optimzer ("adam", "sgd" etc) / scheduler combinations
  # add to "optimizers" parameter in Trainer class
  # ------------------------------------------------------------------------------------
  # optimizer = "adamw" # default in Trainer
  # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_train_steps) # default in trainer
  # optimizer = trial.suggest_categorical("optimizer", ["adamw", "adam", "sgd"])

  # Create the optimizer with the hyperparameter learning rate
#   if optimizer == "adamw":
#       optimizer_obj = AdamW(model.parameters(), lr=learning_rate)
#   elif optimizer == "adam":
#       optimizer_obj = Adam(model.parameters(), lr=learning_rate)
#   elif optimizer == "sgd":
#       optimizer_obj = SGD(model.parameters(), lr=learning_rate)


#   scheduler = get_linear_schedule_with_warmup(
#     optimizer_obj,
#     num_warmup_steps=trial.suggest_int("num_warmup_steps", 1, 10), # so könnte man diesen Parameter auch tunen, falls es da keine Vorgabewert gibt
#     num_training_steps=trial.suggest_int("num_training_steps", 1, 10) # so könnte man diesen Parameter auch tunen, falls es da keine Vorgabewert gibt
#   )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=AmazonReviewDataset(full_review=np.array(X_train), labels=np.array(y_train), tokenizer=tokenizer, max_len=max_len),
      eval_dataset=AmazonReviewDataset(full_review=np.array(X_val), labels=np.array(y_val), tokenizer=tokenizer, max_len=max_len),
      compute_metrics=compute_metrics,
      # optimizers = (optimizer_obj, scheduler) # uncomment if you want to use another optimizer scheduler than the default in the trainer
  )

  # training, evaluation and prediction loops can be done with one simple line of code:
  trainer.train()
  eval_metrics = trainer.evaluate()
  pred_tuple = trainer.predict(AmazonReviewDataset(full_review=np.array(X_test), labels=np.array(y_test), tokenizer=tokenizer, max_len=max_len))

  return pred_tuple, eval_metrics


In [20]:
pred_tuple, eval_metrics = objective()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Trainer is attempting to log a value of "[0.5059486031532288, 0.4381769299507141, 0.5193440318107605, 0.45599082112312317, 0.5147963762283325, 0.5386120080947876, 0.5005741119384766, 0.6970396637916565, 0.7908027172088623, 0.5475224852561951, 0.5560576319694519, 0.4490280747413635, 0.4909900426864624, 0.6032640337944031, 0.5040342211723328, 0.6165058016777039, 0.4915897846221924, 0.695127010345459, 0.5273452401161194, 0.9461348056793213, 0.34677690267562866, 0.6893736124038696, 0.5115973949432373, 0.5471678376197815, 0.45019346475601196, 0.5690855979919434, 0.43194612860679626, 0.6682547330856323, 0.6363771557807922, 0.5566110014915466, 0.45080843567848206, 0.32908737659454346, 0.6158172488212585, 0.5353230834007263, 0.39774855971336365, 0.49521854519844055, 0.5751415491104126, 0.45900678634643555, 0.3852839469909668, 0.5528672337532043, 0.8336969017982483, 0.5711669325828552, 0.5567879676818848, 0.6845414042472839, 0.7141880393028259, 0.8736634254455566, 0.5656546354293823, 0.71948671

Trainer is attempting to log a value of "[0.4992174208164215, 0.5928914546966553, 0.7941394448280334, 0.6071407794952393, 0.5147024989128113, 0.5688296556472778, 0.645155131816864, 0.8640664219856262, 0.760484516620636, 0.5566940903663635, 0.5023901462554932, 0.5788747668266296, 0.7480923533439636, 0.6369251608848572, 0.7196707129478455, 0.62172931432724, 0.6876585483551025, 0.5411719679832458, 0.6556279063224792, 0.9778923392295837, 0.6282475590705872, 0.7772853374481201, 0.4459964334964752, 0.6915749907493591, 0.42099785804748535, 0.8017295598983765, 0.5916486382484436, 0.7678692936897278, 0.5721132755279541, 0.7169898152351379, 0.5840061902999878, 0.5253461003303528, 0.7703599333763123, 0.49905455112457275, 0.6026275753974915, 0.5387068390846252, 0.770820677280426, 0.4863021969795227, 0.5106825232505798, 0.6739328503608704, 0.8069400787353516, 0.49974074959754944, 0.7625111937522888, 0.8569262623786926, 0.7345871329307556, 0.892918050289154, 0.5951150059700012, 0.9089444875717163, 0

In [ ]:
pred_tuple

In [23]:
eval_metrics = pd.DataFrame(eval_metrics)

In [25]:
eval_metrics

,eval_loss,eval_accuracy,eval_recall_score,eval_precision_score,eval_f1_score,eval_highest_probability,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.993239,0.587889,0.587889,0.587889,0.587889,0.499217,48.3144,186.28,11.653,2.0
1,0.993239,0.587889,0.587889,0.587889,0.587889,0.592891,48.3144,186.28,11.653,2.0
2,0.993239,0.587889,0.587889,0.587889,0.587889,0.794139,48.3144,186.28,11.653,2.0
3,0.993239,0.587889,0.587889,0.587889,0.587889,0.607141,48.3144,186.28,11.653,2.0
4,0.993239,0.587889,0.587889,0.587889,0.587889,0.514702,48.3144,186.28,11.653,2.0
...,...,...,...,...,...,...,...,...,...,...
8995,0.993239,0.587889,0.587889,0.587889,0.587889,0.816794,48.3144,186.28,11.653,2.0
8996,0.993239,0.587889,0.587889,0.587889,0.587889,0.862114,48.3144,186.28,11.653,2.0
8997,0.993239,0.587889,0.587889,0.587889,0.587889,0.449996,48.3144,186.28,11.653,2.0
8998,0.993239,0.587889,0.587889,0.587889,0.587889,0.688758,48.3144,186.28,11.653,2.0


In [24]:
eval_metrics.to_csv("prediction/bert_balanced_eval_metrics.csv")

# Inspect sentiment predictions

In [26]:
predicted_labels = np.argmax(pred_tuple[0], axis=1)
actual_labels = pred_tuple[1]
print(predicted_labels)
print(actual_labels)

[3 3 4 ... 4 2 4]
[4 4 4 ... 4 2 4]


In [27]:
predictions_df = pd.DataFrame(np.argmax(pred_tuple[0], axis=1))
actual_df = pd.DataFrame(pred_tuple[1])
highest_probability = pd.DataFrame(pred_tuple[2]['test_highest_probability'])

In [28]:
X_test = pd.DataFrame(X_test)

In [29]:
combined_df = pd.concat([X_test, predictions_df, actual_df, highest_probability], axis=1, keys=['Full Review', 'Predictions', 'Actual Labels', 'Highest Probability'])

In [30]:
combined_df.columns = ['Full Review', 'Predicted', 'Actual', 'Highest Probability']

In [31]:
combined_df

,Full Review,Predicted,Actual,Highest Probability
0,"very fun game, good graphics and realistic dri...",3,4,0.566855
1,Awesome Case I really like this case. When I w...,3,4,0.760029
2,Great game,4,4,0.665463
3,I ordered this by mistake thinking that I was ...,4,4,0.558502
4,The game is very generic but fun my husband lo...,3,3,0.566385
...,...,...,...,...
165478,Works perfectly.,4,4,0.934820
165479,Great! Exactly as described! Perfect for my Xb...,4,4,0.962807
165480,Works perfectly.,4,4,0.934820
165481,Play it for the fun not the story. This is a f...,2,2,0.709023


In [33]:
combined_df.to_csv("prediction/bert_balanced_pred_analysis.csv")

In [32]:
# all reviews where BERT fails to predict the star rating sorted by highest probability i.e. confidence of the model
combined_df[combined_df['Predicted'] != combined_df['Actual']].sort_values(by=['Highest Probability'])

,Full Review,Predicted,Actual,Highest Probability
7770,4/5 why. When I first got the hard drive ... 4...,4,3,0.221724
37006,Kids love it - trips malware warnings,2,3,0.222201
115455,Finally Found a Decent Android Controller! Had...,0,4,0.225114
40043,It looks just like the image Real genuine Micr...,2,4,0.226234
123344,"brutal What's better than killing monsters, ma...",4,3,0.230409
...,...,...,...,...
111963,Piece of s*** didn't work on Christmas day an...,0,2,0.974499
46985,Waste of money. Totally did not work.,0,1,0.974880
9716,Wasted birthday money on this defective game M...,0,3,0.975563
534,Huge waste of money The game rarely works. DO...,0,1,0.975779


In [62]:
combined_df[combined_df['Highest Probability'] < 0.90].sort_values(by=['Highest Probability'])

,Full Review,Predicted,Actual,Highest Probability
2544,... really wasn't Wat I thought it would be no...,4,0,0.229864
6782,I hope you really like watching loading screen...,2,2,0.241429
4119,Guy at Best Buy says that it's common Not sure...,2,2,0.258818
461,"great game, day off from work ruined from late...",0,3,0.260622
5773,Don't read the one star reviews. Watch gamepla...,1,4,0.260807
...,...,...,...,...
1327,Perfect I got this to keep my xbox cooler and ...,4,4,0.899374
7633,... CAN BE SO VERY VERY EVIL IN GAME ITS LIKE ...,4,4,0.899440
1673,Was an easy apply and does the job it says Was...,4,4,0.899507
6135,"Good stuff Brand new, cheaper than the store.<...",4,4,0.899509


# BERT With Stratified Data

## Hyperparameter Tuning

In [14]:
import optuna

def objective(trial):
  # BERT model to tune
  # ------------------------------
  # PRE_TRAINED_MODEL_NAME = model

  # Hyperparameters
  num_train_epochs = trial.suggest_int("num_train_epochs", 2, 4)
  per_device_train_batch_size = trial.suggest_int("train_batch_size", 8, 32)
  per_device_eval_batch_size = trial.suggest_int("eval_batch_size", 3, 32)
  learning_rate = trial.suggest_float("learning_rate", 0.000001, 0.0001)

  training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=num_train_epochs, per_device_train_batch_size=per_device_train_batch_size, per_device_eval_batch_size=per_device_eval_batch_size, learning_rate=learning_rate)
  max_len = 256 # max number of token inputs

  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # bert outputs a raw representation of its confidence as "logits"
    probabilities = torch.nn.functional.softmax(torch.from_numpy(logits), dim=-1) # extract relative probabilites of class
    predicted_probabilities = torch.max(probabilities, axis=-1).values # get highest relative probability
    return { 'accuracy': accuracy_score(labels, predictions), 
            'recall_score': recall_score(labels, predictions, average='micro'), 
            'precision_score': precision_score(labels, predictions, average='micro'), 
            'f1_score': f1_score(labels, predictions, average='micro'), 
            #'highest_probability': predicted_probabilities 
            }

  model = BertForSequenceClassification.from_pretrained(
      PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
      num_labels = 5, # The number of output labels, in our case it's multi-class tasks with classes=5  
      output_attentions = False, # Whether the model returns attentions weights.
      output_hidden_states = False, # Whether the model returns all hidden-states.
  )

  # Experimentation with different optimzer ("adam", "sgd" etc) / scheduler combinations
  # add to "optimizers" parameter in Trainer class
  # ------------------------------------------------------------------------------------
  # optimizer = "adamw" # default in Trainer
  # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_train_steps) # default in trainer
  # optimizer = trial.suggest_categorical("optimizer", ["adamw", "adam", "sgd"])

  # Create the optimizer with the hyperparameter learning rate
#   if optimizer == "adamw":
#       optimizer_obj = AdamW(model.parameters(), lr=learning_rate)
#   elif optimizer == "adam":
#       optimizer_obj = Adam(model.parameters(), lr=learning_rate)
#   elif optimizer == "sgd":
#       optimizer_obj = SGD(model.parameters(), lr=learning_rate)


#   scheduler = get_linear_schedule_with_warmup(
#     optimizer_obj,
#     num_warmup_steps=trial.suggest_int("num_warmup_steps", 1, 10), # so könnte man diesen Parameter auch tunen, falls es da keine Vorgabewert gibt
#     num_training_steps=trial.suggest_int("num_training_steps", 1, 10) # so könnte man diesen Parameter auch tunen, falls es da keine Vorgabewert gibt
#   )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=AmazonReviewDataset(full_review=np.array(X_train_stratified), labels=np.array(y_train_stratified), tokenizer=tokenizer, max_len=max_len),
      eval_dataset=AmazonReviewDataset(full_review=np.array(X_val_stratified), labels=np.array(y_val_stratified), tokenizer=tokenizer, max_len=max_len),
      compute_metrics=compute_metrics,
      # optimizers = (optimizer_obj, scheduler) # uncomment if you want to use another optimizer scheduler than the default in the trainer
  )

  # training, evaluation and prediction loops can be done with one simple line of code:
  trainer.train()
  eval_metrics = trainer.evaluate()
  # pred_tuple = trainer.predict(AmazonReviewDataset(full_review=np.array(X_test), labels=np.array(y_test), tokenizer=tokenizer, max_len=max_len))

  #return pred_tuple, eval_metrics
  # Der Ausgabewert der Funktion darf nur ein numerischer Wert sein, also kein Tuple oder auch keine Liste von Metriken
  return eval_metrics['eval_f1_score']


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10) # je nachdem wie lange ein trial durchläuft kann man die Zahl auch erhöhen

# so kann man dann auf die trials zugreifen und die Hyperparameter mit dem jeweiligen Ergebnis anschauen
for trial in study.trials:
  hyperparams = trial.params
  result = trial.value
  print(f"Hyperparameters: {hyperparams}, Result: {result}")

# oder man greift direkt auf die erfolgreichste Hyperparameter Kombination zu
print(f"Best set of hyperparameters: {study.best_trial.params}, Result: {study.best_trial.value}")

[I 2023-05-25 08:11:06,277] A new study created in memory with name: no-name-c61b276c-6c0a-4283-9691-e6a24e559712
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClas

Epoch,Training Loss,Validation Loss,Accuracy,Recall Score,Precision Score,F1 Score
1,0.717600,0.719995,0.735111,0.735111,0.735111,0.735111
2,0.603100,0.752935,0.754333,0.754333,0.754333,0.754333
3,0.485400,0.704880,0.757000,0.757000,0.757000,0.757000
4,0.339900,0.849484,0.751778,0.751778,0.751778,0.751778


[I 2023-05-25 08:36:07,265] Trial 0 finished with value: 0.7517777777777778 and parameters: {'num_train_epochs': 4, 'train_batch_size': 14, 'eval_batch_size': 12, 'learning_rate': 9.794260196110003e-05}. Best is trial 0 with value: 0.7517777777777778.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the ch

Epoch,Training Loss,Validation Loss,Accuracy,Recall Score,Precision Score,F1 Score
1,0.735300,0.665887,0.743667,0.743667,0.743667,0.743667
2,0.537200,0.665704,0.763778,0.763778,0.763778,0.763778
3,0.363100,0.742954,0.759556,0.759556,0.759556,0.759556


[I 2023-05-25 08:53:47,956] Trial 1 finished with value: 0.7595555555555554 and parameters: {'num_train_epochs': 3, 'train_batch_size': 25, 'eval_batch_size': 8, 'learning_rate': 7.67150966525082e-05}. Best is trial 1 with value: 0.7595555555555554.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the chec

...

Epoch,Training Loss,Validation Loss,Accuracy,Recall Score,Precision Score,F1 Score
1,0.707500,0.649534,0.746889,0.746889,0.746889,0.746889
2,0.541500,0.638214,0.769889,0.769889,0.769889,0.769889
3,0.414400,0.721988,0.765556,0.765556,0.765556,0.765556
4,0.286400,0.835332,0.753333,0.753333,0.753333,0.753333


[I 2023-05-25 11:09:31,228] Trial 8 finished with value: 0.7533333333333333 and parameters: {'num_train_epochs': 4, 'train_batch_size': 27, 'eval_batch_size': 11, 'learning_rate': 4.0993531776729285e-05}. Best is trial 7 with value: 0.7663333333333332.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the c

Epoch,Training Loss,Validation Loss,Accuracy,Recall Score,Precision Score,F1 Score
1,0.713200,0.620276,0.764222,0.764222,0.764222,0.764222
2,0.525100,0.628993,0.767222,0.767222,0.767222,0.767222


[I 2023-05-25 11:21:32,969] Trial 9 finished with value: 0.7672222222222224 and parameters: {'num_train_epochs': 2, 'train_batch_size': 28, 'eval_batch_size': 12, 'learning_rate': 2.7700723449416458e-05}. Best is trial 9 with value: 0.7672222222222224.


Hyperparameters: {'num_train_epochs': 4, 'train_batch_size': 14, 'eval_batch_size': 12, 'learning_rate': 9.794260196110003e-05}, Result: 0.7517777777777778
Hyperparameters: {'num_train_epochs': 3, 'train_batch_size': 25, 'eval_batch_size': 8, 'learning_rate': 7.67150966525082e-05}, Result: 0.7595555555555554
Hyperparameters: {'num_train_epochs': 4, 'train_batch_size': 22, 'eval_batch_size': 20, 'learning_rate': 2.9304254810680445e-05}, Result: 0.7575555555555555
Hyperparameters: {'num_train_epochs': 2, 'train_batch_size': 22, 'eval_batch_size': 23, 'learning_rate': 7.514097443993254e-05}, Result: 0.7616666666666667
Hyperparameters: {'num_train_epochs': 3, 'train_batch_size': 27, 'eval_batch_size': 31, 'learning_rate': 9.722230724259963e-05}, Result: 0.7483333333333333
Hyperparameters: {'num_train_epochs': 3, 'train_batch_size': 10, 'eval_batch_size': 15, 'learning_rate': 3.333624838392188e-05}, Result: 0.7617777777777778
Hyperparameters: {'num_train_epochs': 4, 'train_batch_size': 29, 

In [17]:
# Save the DataFrame to a CSV file
best_params = pd.DataFrame([study.best_trial])
best_params.to_csv('hyperparameter_tuning/bert_stratified_best_params.csv', index=False)



In [19]:
# Get the trials and their results
trials = study.trials
results = [{'trial_number': trial.number, 'value': trial.value, 'params': trial.params} for trial in trials]

# Create a DataFrame from the results
results = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results.to_csv('hyperparameter_tuning/bert_stratified_hyperparameter_results.csv', index=False)

## Training

Training with best parameters from hyperparameter tuning.

In [16]:
def objective():
  # BERT model to tune
  # ------------------------------
  # PRE_TRAINED_MODEL_NAME = model

  # Hyperparameters
  num_train_epochs = 2
  per_device_train_batch_size = 28
  per_device_eval_batch_size = 12
  learning_rate = 2.7700723449416458e-05

  training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=num_train_epochs, per_device_train_batch_size=per_device_train_batch_size, per_device_eval_batch_size=per_device_eval_batch_size, learning_rate=learning_rate)
  max_len = 256 # max number of token inputs

  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # bert outputs a raw representation of its confidence as "logits"
    probabilities = torch.nn.functional.softmax(torch.from_numpy(logits), dim=-1) # extract relative probabilites of class
    predicted_probabilities = torch.max(probabilities, axis=-1).values # get highest relative probability
    return { 'accuracy': accuracy_score(labels, predictions), 
            'recall_score': recall_score(labels, predictions, average='micro'), 
            'precision_score': precision_score(labels, predictions, average='micro'), 
            'f1_score': f1_score(labels, predictions, average='micro'), 
            'highest_probability': predicted_probabilities.tolist() 
            }

  model = BertForSequenceClassification.from_pretrained(
      PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
      num_labels = 5, # The number of output labels, in our case it's multi-class tasks with classes=5  
      output_attentions = False, # Whether the model returns attentions weights.
      output_hidden_states = False, # Whether the model returns all hidden-states.
  )

  # Experimentation with different optimzer ("adam", "sgd" etc) / scheduler combinations
  # add to "optimizers" parameter in Trainer class
  # ------------------------------------------------------------------------------------
  # optimizer = "adamw" # default in Trainer
  # scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_train_steps) # default in trainer
  # optimizer = trial.suggest_categorical("optimizer", ["adamw", "adam", "sgd"])

  # Create the optimizer with the hyperparameter learning rate
#   if optimizer == "adamw":
#       optimizer_obj = AdamW(model.parameters(), lr=learning_rate)
#   elif optimizer == "adam":
#       optimizer_obj = Adam(model.parameters(), lr=learning_rate)
#   elif optimizer == "sgd":
#       optimizer_obj = SGD(model.parameters(), lr=learning_rate)


#   scheduler = get_linear_schedule_with_warmup(
#     optimizer_obj,
#     num_warmup_steps=trial.suggest_int("num_warmup_steps", 1, 10), # so könnte man diesen Parameter auch tunen, falls es da keine Vorgabewert gibt
#     num_training_steps=trial.suggest_int("num_training_steps", 1, 10) # so könnte man diesen Parameter auch tunen, falls es da keine Vorgabewert gibt
#   )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=AmazonReviewDataset(full_review=np.array(X_train_stratified), labels=np.array(y_train_stratified), tokenizer=tokenizer, max_len=max_len),
      eval_dataset=AmazonReviewDataset(full_review=np.array(X_val_stratified), labels=np.array(y_val_stratified), tokenizer=tokenizer, max_len=max_len),
      compute_metrics=compute_metrics,
      # optimizers = (optimizer_obj, scheduler) # uncomment if you want to use another optimizer scheduler than the default in the trainer
  )

  # training, evaluation and prediction loops can be done with one simple line of code:
  trainer.train()
  eval_metrics = trainer.evaluate()
  pred_tuple = trainer.predict(AmazonReviewDataset(full_review=np.array(X_test), labels=np.array(y_test), tokenizer=tokenizer, max_len=max_len))

  return pred_tuple, eval_metrics


In [17]:
pred_tuple, eval_metrics = objective()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Trainer is attempting to log a value of "[0.7562972903251648, 0.8800542950630188, 0.9604126214981079, 0.9210721850395203, 0.4699561297893524, 0.7662039995193481, 0.8653278946876526, 0.791911244392395, 0.7214546203613281, 0.9396162033081055, 0.8953558802604675, 0.9105391502380371, 0.9087538719177246, 0.400982141494751, 0.455609530210495, 0.8145990967750549, 0.46094372868537903, 0.6165570616722107, 0.902554988861084, 0.916714608669281, 0.473934143781662, 0.809123694896698, 0.9656734466552734, 0.9607172012329102, 0.8643137812614441, 0.3714584410190582, 0.9185320734977722, 0.45091062784194946, 0.8781396150588989, 0.9396162033081055, 0.8231409192085266, 0.6184417605400085, 0.942468523979187, 0.9580531716346741, 0.9537529349327087, 0.6762880682945251, 0.9514944553375244, 0.9030046463012695, 0.26091858744621277, 0.7642611265182495, 0.8987863063812256, 0.9602683782577515, 0.5396977066993713, 0.4479331374168396, 0.7470182180404663, 0.5273842215538025, 0.9495123028755188, 0.5051767826080322, 0.4

Trainer is attempting to log a value of "[0.7547562122344971, 0.9120832085609436, 0.9781426191329956, 0.9473307132720947, 0.550046443939209, 0.7212396264076233, 0.8526787757873535, 0.7562962174415588, 0.43536290526390076, 0.9408384561538696, 0.8881537914276123, 0.9513970613479614, 0.9252727627754211, 0.3287222683429718, 0.653455913066864, 0.8698752522468567, 0.6065760254859924, 0.5975871086120605, 0.9355391263961792, 0.9532766938209534, 0.5920057892799377, 0.8608233332633972, 0.9767950177192688, 0.9747762084007263, 0.9021354913711548, 0.3106862008571625, 0.9628304243087769, 0.5479251146316528, 0.8709917068481445, 0.9408384561538696, 0.8395673632621765, 0.7471278309822083, 0.9283331632614136, 0.9615399241447449, 0.9653962254524231, 0.6512324810028076, 0.9336455464363098, 0.9380002021789551, 0.46139049530029297, 0.8677002787590027, 0.944024384021759, 0.9656421542167664, 0.6316103935241699, 0.4985756576061249, 0.6861014366149902, 0.8667821884155273, 0.972282350063324, 0.46268847584724426,

In [19]:
eval_metrics = pd.DataFrame(eval_metrics)
eval_metrics.to_csv("prediction_result/bert_stratified_eval_metrics.csv")

In [27]:
eval_metrics

,eval_loss,eval_accuracy,eval_recall_score,eval_precision_score,eval_f1_score,eval_highest_probability,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,0.631038,0.766111,0.766111,0.766111,0.766111,0.754756,45.6744,197.047,16.421,2.0
1,0.631038,0.766111,0.766111,0.766111,0.766111,0.912083,45.6744,197.047,16.421,2.0
2,0.631038,0.766111,0.766111,0.766111,0.766111,0.978143,45.6744,197.047,16.421,2.0
3,0.631038,0.766111,0.766111,0.766111,0.766111,0.947331,45.6744,197.047,16.421,2.0
4,0.631038,0.766111,0.766111,0.766111,0.766111,0.550046,45.6744,197.047,16.421,2.0
...,...,...,...,...,...,...,...,...,...,...
8995,0.631038,0.766111,0.766111,0.766111,0.766111,0.808031,45.6744,197.047,16.421,2.0
8996,0.631038,0.766111,0.766111,0.766111,0.766111,0.831318,45.6744,197.047,16.421,2.0
8997,0.631038,0.766111,0.766111,0.766111,0.766111,0.906883,45.6744,197.047,16.421,2.0
8998,0.631038,0.766111,0.766111,0.766111,0.766111,0.932114,45.6744,197.047,16.421,2.0


## Inspect sentiment prediction

In [20]:
predicted_labels = np.argmax(pred_tuple[0], axis=1)
actual_labels = pred_tuple[1]
print(predicted_labels)
print(actual_labels)

[4 4 4 ... 4 3 4]
[4 4 4 ... 4 2 4]


In [21]:
predictions_df = pd.DataFrame(np.argmax(pred_tuple[0], axis=1))
actual_df = pd.DataFrame(pred_tuple[1])
highest_probability = pd.DataFrame(pred_tuple[2]['test_highest_probability'])

In [22]:
X_test = pd.DataFrame(X_test)

In [23]:
combined_df = pd.concat([X_test, predictions_df, actual_df, highest_probability], axis=1, keys=['Full Review', 'Predictions', 'Actual Labels', 'Highest Probability'])

In [24]:
combined_df.columns = ['Full Review', 'Predicted', 'Actual', 'Highest Probability']

In [25]:
combined_df.to_csv("prediction_result/bert_stratified_pred_analysis.csv")

In [26]:
# all reviews where BERT fails to predict the star rating sorted by highest probability i.e. confidence of the model
combined_df[combined_df['Predicted'] != combined_df['Actual']].sort_values(by=['Highest Probability'])

,Full Review,Predicted,Actual,Highest Probability
125260,The game isn't bad for a bunch of unrelated mu...,1,2,0.225085
85025,... perfectly for exactly 2 hours before break...,2,0,0.237148
24932,Just be careful Was advertised as &#34;Like ne...,2,1,0.240956
15551,I bought this on sale and I am so glad I did I...,2,1,0.241919
134600,you have to take off the motion plus to charg...,0,1,0.248264
...,...,...,...,...
72014,BEST Video gaming System Ever created!!!! Woul...,4,3,0.986657
37136,All Geared Up Can't wait to play to one of the...,4,3,0.987151
159307,Awesome!! Got this a day earlier than expected...,4,3,0.987763
69886,This is amazing! Visit http://psncodesonlinef...,4,3,0.987820


# Prediction from Pre-Trained and Fine-Tuned BERT Model



In [41]:
from transformers import pipeline
from transformers import AutoTokenizer

In [30]:
sentiment_pipeline = pipeline(model="nlptown/bert-base-multilingual-uncased-sentiment")

In [38]:
X_test_array = np.array(X_test)

In [60]:
X_test = X_test.iloc[:, 0].fillna('')

In [62]:
X_test_list = X_test.tolist()

In [76]:
tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
# Tokenize reviews and truncate/omit exceeding tokens
tokenized_reviews = [tokenizer.encode_plus(
                        review,
                        truncation=True,
                        max_length=510,
                        padding="max_length",
                        return_tensors="pt"
                     ) for review in X_test_list]

In [77]:
# Convert tokenized reviews back to original string representation
text_reviews = [tokenizer.decode(review['input_ids'][0]) for review in tokenized_reviews]

In [ ]:
predictions = sentiment_pipeline(text_reviews)

TO-DOs:

 a) run more epochs? plot the accuracy plot for training & validation set

 b) try larger/smaller batch size

 c) different learning rate/optimizers?

 d) what should be the optimal max_len for our dataset?

 e) how about different pre-trained model? (e.g. bert-base-cased)

 f) Instead of bert, try different transformer-based model (i.e. distilbert, roberta, ...)